In [ ]:
# AutoML을 이용한 ML 구현
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
# suboptmal problem의 iteration 문제는 무시할 것! (suboptimal의 계산횟수를 한정하여 수렴시키겠다는 얘기)
# 궁금하면 max_iter라는 값을 인자로 전달받는 ML 알고리즘을 살펴보세요!
import warnings
warnings.filterwarnings('ignore')

# 필요한 알고리즘을 불러와야 합니다!
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
# Ridge와 Lasso의 hyperparmeter: alpha
# Logistic Regressio의 hyperparameter: penalty와 regularization strength인 C값
from sklearn.svm import SVR
# SVR의 hyperparameter: epsilon, regularization C, gamma, kernel = 'rbf', 'poly', 'sigmoid'
from sklearn.neural_network import MLPRegressor
# hidden_layer_sizes = (100,) , (10, 10, ) 정도만 activation = 'relu', 'logistic'까지만, alpha =0.0001, solver = 'lbfgs', 'adam'까지만

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(중식계 트레이닝 데이터, 중식계 target 값, shuffle=True)
X2_train, X2_test, y2_train, y2_test = train_test_split(석식계 트레이닝 데이터, 석식계 target 값, shuffle=True)

In [ ]:
pipe = Pipeline([('preprocessing', None), ('regressor', LinearRegression())])
# 우선 파이프라인 객체를 만들어줍니다. 구상하는 파이프라인은 전처리 -> 모델적합
# 아시다시피 파이프라인에 전달해야하는 건 steps인데, steps는 tuple을 list로 모아서 전달해야 합니다.
# 'preprocessing'과 'regressor'는 키워드로 제가 임의로 설정한 겁니다.

pre_list = [MinMaxScaler(), StandardScaler(), None]
# 이거는 그냥 계속 반복해서 쓰는 게 귀찮아서 미리 변수에 저장한 거에요!

hyperparam_grid = [
    {"regressor": [LinearRegression()], 'preprocessing': pre_list},
    {"regressor": [Ridge()], 'preprocessing': pre_list,
     "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1]},
    {"regressor": [Lasso()], 'preprocessing': pre_list,
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1]},
    {"regressor": [SVR()], 'preprocessing': pre_list,
    "regressor__epsilon": [0.001, 0.01, 0.1, 1],
    "regressor__C": [0.0001, 0.001, 0.01, 0.1, 1],
    "regressor__gamma": [0.0001, 0.001, 0.01, 0.1, 1]},
    {"regressor": [MLPRegressor()], 'preprocessing': pre_list,
    "regressor__hidden_layer_sizes": [(100,) , (10, 10, )],
    "regressor__activation": ["relu", "logistic"],
    "regressor__solver": ["lbfgs", "adam"],
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1]}

]
# grid는 딕셔너리를 list의 형태로 전달하고 있습니다.
# 각 알고리즘별 하이퍼파라미터를 '__'로 접근하고 있고 반드시 pipe에 써준 keyword랑 조합해서 써줘야 합니다.


# cross-validation을 세부 설정하려고 따로 빼서 선언했습니다.
kfold = KFold(n_splits=5, shuffle=True, random_state=1)


# 중식계를 위해 grid1을, 석식계를 위해 grid1을 만들었습니다.
# 왜 scoring이 neg_mean_absolute_error 일까요? Ans: 음수 붙이면 커질수록 성능이 좋아지니까.
grid1 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)
grid2 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)

In [ ]:
grid1.fit(X1_train, y1_train)
print(grid1.best_estimator_)
print(grid1.best_params_)
print(grid1.best_score_)
print(grid1.score(X1_test, y1_test))

In [ ]:
grid2.fit(X2_train, y2_train)
print(grid2.best_estimator_)
print(grid2.best_params_)
print(grid2.best_score_)
print(grid2.score(X2_test, y2_test))

In [ ]:
a = pd.DataFrame(grid1.best_estimator_.predict(test[lfeature_column]))

In [ ]:
b = pd.DataFrame(grid1.best_estimator_.predict(test[dfeature_column]))

In [ ]:
save = pd.read_csv("sample_submission.csv")
save["중식계"] = a
save["석식계"] = b
save.columns = ["일자", "중식계", "석식계"]
save.to_csv("submit.csv", encoding="CP949", index=False)